# 자연스러운 코드로

- 가변 길이 인수 (순전파)
- 가변 길이 인수 (개선)
- 가변 길이 인수 (역전파)


## 가변 길이 인수 (순전파)

```python
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # Variable 인스턴스로부터 데이터를 꺼낸다.
        ys = self.forward(xs) # forward 메서드에서 구체적인 계산을 수행한다.
        outputs = [Variable(as_array(y)) for y in ys] # 계산된 데이터를 Variable 인스턴스로 다시 감싼다.

        for output in outputs:
            output.set_creator(self) # 원산지 표시를 한다.

        self.inputs = inputs  # 입력 변수를 기억(보관)한다.
        self.outputs = outputs  # 출력 변수를 저장한다.
        return outputs

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

인수와 반환값을 리스트로 변경했다.


### Add 클래스

```python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)  # 반환값을 튜플로 묶는다.
```


In [1]:
# import numpy as np
# from framework.variable import Variable
# from framework.function import Add
# xs = [Variable(np.array(2)), Variable(np.array(3))]
# f = Add()
# ys = f(xs)
# y = ys[0]
# print(y.data)

## 가변 길이 인수 (개선)

### 함수를 사용하기 쉽게 개선

```python
class Function:
  def __call__(self, *inputs):  # *를 붙여 가변 길이 인수로 변경
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)  # 리스트 대신 언패킹하여 넘긴다.
    if not isinstance(ys, tuple):  # 튜플이 아닌 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
```


In [2]:
import numpy as np
from framework.function import Add
from framework.variable import Variable
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [3]:
from framework.function import add
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## 13 가변 길이 인수 (역전파)

### 13.1 가변 기이 인수에 대응한 Add클래스의 역전파

```python
class Add(Function):
    def forward(self, *xs):
        x0, x1 = xs
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy
```


### 13.2 Variable 클래스의 backward 메서드 수정

```python
def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.

            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                x.grad = gx

                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)
```


### 13.3 Square 클래스 구현

```python
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전: x = self.input.data
        gx = 2 * x * gy
        return gx
```


In [4]:
from framework.variable import Variable
from framework.function import add, square
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


## 14 같은 변수 반복 사용

현재는 같은 변수를 반복해서 사용할 경우 결과값이 지랄맞게 나온다.


In [5]:
import numpy as np

class Variable:
    def __init__(self, data: np.ndarray):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')
        self.data = data  # 데이터 ndarray
        self.grad = None  # 미분값을 저장하는 변수 ndarray
        self.creator = None  # 변수의 창조자(creator)를 기억하는 변수
        self.generation = 0  # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다(부모 세대 + 1)

    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input  # 함수의 입력을 가져온다.
    #         x.grad = f.backward(self.grad)  # 함수의 backward 메서드를 호출한다.
    #         x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다.

    def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.
            
            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                x.grad = gx # 같은 변수를 반복 사용할 경우 덮어쓰기 때문에 주의해야 한다.
                # if x.grad is None:
                #     x.grad = gx
                # else:
                #     x.grad = x.grad + gx
                
                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)


# from framework.variable import Variable
from framework.function import add, square

x = Variable(np.array(3.0))
y = add(x, x)

print('y: ', y.data)

y.backward()
print('x.grad: ', x.grad)

y:  6.0
x.grad:  2.0


In [6]:
import numpy as np


class Variable:
    def __init__(self, data: np.ndarray):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')
        self.data = data  # 데이터 ndarray
        self.grad = None  # 미분값을 저장하는 변수 ndarray
        self.creator = None  # 변수의 창조자(creator)를 기억하는 변수
        self.generation = 0  # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다(부모 세대 + 1)

    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input  # 함수의 입력을 가져온다.
    #         x.grad = f.backward(self.grad)  # 함수의 backward 메서드를 호출한다.
    #         x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다.

    def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        functions = [self.creator]

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.
            
            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                # x.grad = gx # 같은 변수를 반복 사용할 경우 덮어쓰기 때문에 주의해야 한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    functions.append(x.creator)

from framework.function import add, square
x = Variable(np.array(3.0))
y = add(x, x)

print('y: ', y.data)

y.backward()
print('x.grad: ', x.grad)

y:  6.0
x.grad:  2.0


In [7]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)

x.grad:  3.0


### 14.3 미분값 재설정


In [8]:
from framework.function import add, square
from framework.variable import Variable

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad: ', x.grad)

y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)


x.grad:  2.0
x.grad:  5.0


In [9]:
from framework.function import add, square
from framework.variable import Variable

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad: ', x.grad)
x.cleargrad() # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print('x.grad: ', x.grad)


x.grad:  2.0
x.grad:  3.0


## 15 복잡한 계산 그래프 (이론)


## 16 복잡한 계산 그래프 (구현)


### 16.1 세대 추가

````python
def __init__(self, data: np.ndarray):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')
        self.data = data  # 데이터 ndarray
        self.grad = None  # 미분값을 저장하는 변수 ndarray
        self.creator = None  # 변수의 창조자(creator)를 기억하는 변수
        self.generation = 0  # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대를 기록한다(부모 세대 + 1)

    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input  # 함수의 입력을 가져온다.
    #         x.grad = f.backward(self.grad)  # 함수의 backward 메서드를 호출한다.
    #         x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다.
```




In [10]:
import numpy as np
from framework.variable import Variable
from framework.function import add, square
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0


### 16.2 세대 순으로 꺼내기


In [11]:
generations = [2, 0, 1, 4, 2]

funcs = []

for g in generations:
    f = Variable(np.array(g))
    if funcs:
        f.set_creator(funcs[-1])
    funcs.append(f)
    
[f.generation for f in funcs]
    

[0, 1, 2, 3, 4]

In [12]:
# 세대수가 가장 큰 함수를 찾는다.
funcs.sort(key=lambda x: x.generation)
max_generation = funcs[-1].generation
f = funcs[-1]
f.generation

4

### 16.3 Variable 클래스의 backword 메서드 추가 구현

```python
def backward(self):
        """재귀에서 반복문으로 변경"""

        if self.grad is None:
            self.grad = np.ones_like(self.data)

        # functions = [self.creator]
        functions = []

        seen_set = set() # 중복을 방지하기 위한 집합(set) 자료구조
        def add_func(f):
            if f not in seen_set:
                functions.append(f)
                seen_set.add(f)
                functions.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while functions:
            f = functions.pop()  # 함수를 가져온다.
            # x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

            # if x.creator is not None:
            #     functions.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.

            gys = [output.grad for output in f.outputs] # 출력변수인 output에서 출력변수의 미분을 가져온다.
            gxs = f.backward(*gys) # 역전파 호출
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 추가 지원
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분을 Variable의 인스턴스 변수 grad에 저장한다.
                # x.grad = gx # 같은 변수를 반복 사용할 경우 덮어쓰기 때문에 주의해야 한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None: # 하나 앞의 함수를 리스트에 추가한다.
                    add_func(x.creator)
```

가장 큰 변화는 새로 추가된 `add_func`함수이다. 이 함수는 함수를 리스트에 추가할 때 중복을 방지하기 위해 집합(set) 자료구조를 사용한다. 그리고 `functions`리스트에 추가할 때는 `functions.sort(key=lambda x: x.generation)`을 사용하여 세대 순으로 정렬한다.

- 감싸는 메서드(backwoard)안에서만 이용한다.
- 감싸는 메서드(backword 메서드)에 정의된 변수(func과seen_set)를 사용한다.


In [13]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0


결과의 미분은 64.0이다. 수식으로 확인하면 계산그래프는 $y=(x^2)^2 + (x^2)^2$을 미분하는 문제이다. 이때 $y'=8x^3$이므로 $x=2.0$일 때 미분은 64.0이다.


## 17 메모리 관리와 순환 참조

### 17.1 메모리 관리

파이썬에서 메모리 관리하는 방법은 다음과 같다.
* 참조 카운트가 0이 되는 순간 객체는 삭제된다.
* 불필요한 객체는 즉시 삭제된다. (Garbage Collection)

### 17.2 참조 카운트 방식의 메모리 관리

모든 객체는 참조카운트가 0인 상태로 생성된다.

다음의 경우 참조 카운트가 증가한다.
* 대입 연산자를 사용할 때
* 함수의 인수로 전달할 때
* 컨테이너에 추가할 때
```python
class obj:
  pass

def f(x):
  print(x)

a = obj() # 변수에 대입: 참조 카운트 1
f(a) # 함수의 인수: 참조 카운트 2
# 함수가 종료되면 참조 카운트 1
a = None # 참조 카운트 0
```

### 17.3 순환 참조

```python

a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None
```

3개의 객체가 서로를 참조하고 있기 때문에 참조 카운트가 0이 되지 않는다. 이러한 경우 파이썬은 참조 카운트 방식으로는 메모리를 해제할 수 없다. 이러한 경우 파이썬은 가비지 컬렉션을 사용하여 메모리를 해제한다.

### 17.4 weakref 모듈

파이썬에서는 weakref.ref 함수를 사용하여 약한 참조(weak reference)를 만들 수 있다. 약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조한다.

```python
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

b() # array([1, 2, 3])
```

In [14]:
import weakref

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    
    self.generation = max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = [weakref.ref(output) for output in outputs] # 출력변수를 약한 참조로 가지기
    return outputs if len(outputs) > 1 else outputs[0]

In [16]:
from framework.variable import Variable
from framework.function import Function, Square

for i in range(10):
  x = Variable(np.random.randn(10000))
  y = square(square(square(x)))

## 18 메모리 절약 모드

### 18.1 필요 없는 미분값 삭제

In [19]:
from framework.config import using_config
from framework.variable import Variable
from framework.function import square

with using_config('enable_backprop', False):
  x = Variable(np.array(2.0))
  y = square(x)

y.backward()

In [3]:
import numpy as np
from framework.config import no_grad
from framework.variable import Variable
from framework.function import square

with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)

## 19 변수 사용성 개선

### 19.1 변수 이름 지정

변수에 이름을 붙여줄 수 있도록 설정한다.

```python
class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError(f"{type(data)} is not supported")
    self.data = data
    self.name = name # 이름 추가
    self.creator = None
    self.generation = 0
```

### 19.2 ndarray 인스턴스 변수

### 19.3 len함수와 print함수
